# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [14]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import torch.optim as optim
from torch.autograd import Variable
import cv2

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [15]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [19]:
class SimpleNet(nn.Module):
# TODO:define model

    def __init__(self):
        super(SimpleNet,self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(stride=2,kernel_size=2))
 
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(14*14*128,1024),
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 0.5),
            torch.nn.Linear(1024,10)
        )
 
    def forward(self, x):
        x = self.conv1(x)
        x = x.view(-1,14*14*128)
        x = self.dense(x)
        return x

# TODO:define loss function and optimiter
model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

Next, we can start to train and evaluate!

In [32]:
# train and evaluate
 
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    running_corrrect = 0
    print("Epoch  {}/{}".format(epoch, NUM_EPOCHS))
    print("-"*10)
    for data in train_loader:
        X_train , y_train = data
        X_train,y_train = Variable(X_train),Variable(y_train)
        outputs = model(X_train)
        _,pred = torch.max(outputs.data,1)
        optimizer.zero_grad()
        loss = criterion(outputs,y_train)
 
        loss.backward()
        optimizer.step()
        running_loss += loss
        running_corrrect += torch.sum(pred == y_train.data)
        
    testing_correct = 0
    testing_loss = 0.0
    for data in test_loader:
        X_test,y_test = data
        X_test,y_test = Variable(X_test),Variable(y_test)
        outputs = model(X_test)
        _,pred = torch.max(outputs.data,1)
        optimizer.zero_grad()
        loss2 = criterion(outputs,y_test)
        
        loss2.backward()
        optimizer.step()
        testing_loss += loss2
        testing_correct += torch.sum(pred == y_test.data)
 
    print("Loss is {:.4f}, Train Accuracy is:{:.4f}%,Test Accuracy is:{:.4f}%"
          .format(running_loss/len(train_dataset),100*running_corrrect.item()/len(train_dataset),
                  100*testing_correct.item()/len(test_dataset)))
     


Epoch  0/10
----------
Loss is 0.0001, Train Accuracy is:99.6667%,Test Accuracy is:99.2900%
Epoch  1/10
----------
Loss is 0.0001, Train Accuracy is:99.6633%,Test Accuracy is:99.5200%
Epoch  2/10
----------
Loss is 0.0001, Train Accuracy is:99.6950%,Test Accuracy is:99.6400%
Epoch  3/10
----------
Loss is 0.0000, Train Accuracy is:99.7367%,Test Accuracy is:99.7400%
Epoch  4/10
----------
Loss is 0.0000, Train Accuracy is:99.7350%,Test Accuracy is:99.7400%
Epoch  5/10
----------
Loss is 0.0000, Train Accuracy is:99.7417%,Test Accuracy is:99.8000%
Epoch  6/10
----------
Loss is 0.0000, Train Accuracy is:99.7717%,Test Accuracy is:99.8200%
Epoch  7/10
----------
Loss is 0.0000, Train Accuracy is:99.7600%,Test Accuracy is:99.8000%
Epoch  8/10
----------
Loss is 0.0000, Train Accuracy is:99.7933%,Test Accuracy is:99.8000%
Epoch  9/10
----------
Loss is 0.0000, Train Accuracy is:99.8067%,Test Accuracy is:99.8400%


#### Q5:
Please print the training and testing accuracy.